# Deep Petro

## Preâmbulo

O código abaixo consiste dos imports comuns. Além do mais, configuramos as imagens para ficar de um tamanho aceitável e criamos algumas funções auxiliares. No geral, você pode ignorar a próxima célula.

In [ ]:
# !nvcc --version
# !pip install mxnet-cu100==1.4.1

In [ ]:
# -*- coding: utf8

import matplotlib.pyplot as plt

import mxnet as mx
import mxnet.ndarray as nd

import numpy as np

plt.rcParams['figure.figsize']  = (18, 10)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4

In [ ]:
plt.ion()

plt.style.use('seaborn-colorblind')
plt.rcParams['figure.figsize']  = (12, 8)

## Aula 03 - Features de Imagem e Regressão Logística

### Preparando o dataset: CIFAR-10

O CIFAR-10 é um dataset de pequenas imagens ($32 \times 32 \times 3$) que contemplam 10 classes de objetos. É considerado de baixa dificuldade por se tratarem de classes muito distintas, sem sobreposição. 

A API de dados do gluon contém alguns datasets simples, dentre eles o CIFAR-10. Para ter acesso a esses datasets, basta executar o seguinte import:
```python
from mxnet.gluon import data as gdata
```
A função **```gdata.CIFAR10()```** permite baixar as amostras de treino e teste setando o parâmetro booleano **```train```**,  bem como aplicar transformações aos dados ao criar uma função customizada **```transform(data, label)```**. <br>

O objetivo é alimentar um modelo de regressão logística com as imagens do CIFAR-10, portanto precisamos transformá-las em um vetor bidimensional $x \in R^{n \times d}$, sendo $n$ a quantidade de amostras e $d$ a dimensionalidade da entrada. Para tal, nossa função customizada vai achatar as imagens, de modo que $d = 32 \times 32 \times 3 = 3072$. **A entrada será composta pelas intensidades dos pixels nos 3 canais de cor.**

In [ ]:
from mxnet.gluon import data as gdata

Função customizada para criar o nosso vetor achatado de pixels. Note como a mesma simplemesmente converte um tensor (32, 32, 3). 32 x 32 pixels, 3 canais de cores, em um vetor. Dividimos por 255 para deixar as imagens em escala 0-1.

In [ ]:
def transform(data, label):
    return data.astype('float32').reshape((32*32*3))/255, label.astype('float32')

Carregando treino e teste do CIFAR-10, aplicando a função customizada. O Gluon tem classes prontas para vários datasets clássicos.

In [ ]:
cifar_train = gdata.vision.CIFAR10(root='.', train=True, transform=transform)
cifar_test = gdata.vision.CIFAR10(root='.', train=False, transform=transform)

### Dividindo os dados em imagens (X) e rótulos (Y)

Para propósito didático, reduziremos a quantidade de classes e amostras do dataset. Trabalharemos com apenas 5 classes

In [ ]:
X_train = []
Y_train = []
for x_i, y_i in cifar_train:
    if y_i < 5:                    # pega apenas as primeiras 5 classes
        X_train.append(x_i.asnumpy())
        Y_train.append(y_i)
X_train = np.array(X_train[::5])
Y_train = np.array(Y_train[::5])  # amostra elementos de 5 em 5

In [ ]:
X_test = []
Y_test = []
for x_i, y_i in cifar_test:
    if y_i < 5:                  # pega apenas as primeiras 5 classes
        X_test.append(x_i.asnumpy())
        Y_test.append(y_i)
X_test = np.array(X_test[::5])
Y_test = np.array(Y_test[::5])  # amostra elementos de 5 em 5

In [ ]:
print(len(X_train))

In [ ]:
print(len(X_test))

Visualizando amostras aleatórias do CIFAR-10

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=20, figsize=(20, 3))
for i in range(20):
    axs[i].imshow(cifar_test[np.random.choice(len(cifar_test))][0].asnumpy().reshape((32,32,3)))
    axs[i].set_xticks([])
    axs[i].set_yticks([])

## Regressão Logística com Scikit-Learn

A biblioteca do Scikit-Learn implementa os principais métodos de aprendizado de máquina. A regressão logística que implementamos do zero, pode ser utilizada com alguns comandos curtos dessa biblioteca. Começamos com o import:
```python
from sklearn.linear_model import LogisticRegression
```
Isso vai nos permitir invocar a implementação da regressão logística, bem como treiná-la e testá-la nos nossos dados. Inicializamos o modelo de regressão logística com alguns parâmetros padrão, como o *solver* para otimizar o problema, e o parâmetro *multi_class* que vai definir como minimizar a função de perda.

Para treinar o modelo, basta invocar a função **```clf.fit(dados, rótulos)```**, sendo ```clf``` o seu classificador inicializado. As duas principais formas de testar o seu modelo são:
 - Predizendo rótulos de novos dados: **```clf.predict(dados)```**
 - Calculando a acurácia de predição para um conjunto de dados e rótulos: **```clf.score(dados, rótulos)```**

In [ ]:
import time
from sklearn.linear_model import LogisticRegression

Inicializando um modelo de regressão logística. A logística clássica não tem modo multiclasse. Temos que fazer one-vs-the-rest. Caso mude para `multi_class='multinomial'` a bibliotca faz softmax

In [ ]:
clf = LogisticRegression(random_state=0, solver='liblinear', multi_class='ovr')

In [ ]:
print("Treinando modelo de regressão logística")
start = time.time() ### Tempo inicial
clf.fit(X_train, Y_train)
end = time.time()   ### Tempo final
print("%.2f segundos para treinar"  % (end-start) )

print("Testando..")
start = time.time() ### Tempo inicial
acc = clf.score(X_test, Y_test)
end = time.time()   ### Tempo final
print("%.2f segundos para testar" % (end-start) )

print("\nAcurácia de classificação: %.2f\n" % (acc * 100.) )

## Features de Imagem

Na célula anterior, executamos uma classificação com um modelo de regressão logística, alimentando como entrada um vetor de intensidades de pixels. 

**<center>O que acontece se em vez disso a entrada do modelo sejam features de imagem?</center>**

Abaixo está implementada a função **```extract_features(X)```** que, dadas as imagens do CIFAR-10, extrai 4 diferentes features de imagem: HOG, Dense Daisy, Local Binary Pattern, Color Histogram. 

**Seu trabalho é** escolher a melhor feature (ou combinação de features) para classificar as imagens do dataset. Sinta-se livre para experimentar cada feature individualmente ou combiná-las, basta apenas preencher a lista de features criada dentro da função. Atenção para alguns apectos:

- Desempenho da feature em termos de acurácia. Por que algumas são melhores que outras?
- Tempo de treinamento
- Dimensionalidade da feature (impacta no tempo de treinamento);
- A combinação de diferentes features pode aprimorar **ou não** a performance de classificação (por que?);

In [ ]:
from skimage.color import rgb2gray
from skimage.exposure import histogram

from skimage.feature import daisy
from skimage.feature import hog
from skimage.feature import local_binary_pattern as lbp

In [ ]:
def extract_features(X):
    feats = []
    for x in X:
        ## HOG Features
        fd_hog = hog(x.reshape((32,32,3)),
                     orientations=12,
                     pixels_per_cell=(8,8),
                     cells_per_block=(1, 1),
                     multichannel=True)

        ## Dense Daisy
        fd_daisy = daisy(rgb2gray(x.reshape((32,32,3))),
                         step=180,
                         radius=8,
                         rings=2,
                         histograms=6,
                         orientations=8)

        ## Local Binary Pattern
        fd_lbp = lbp(rgb2gray(x.reshape((32,32,3))),
                     8, 1,
                     method='uniform')

        ## Color Histogram
        x = x.reshape((32,32,3))
        fd_hist = [histogram(x[:,:,0], nbins=10),
                   histogram(x[:,:,1], nbins=10),
                   histogram(x[:,:,2], nbins=10)]

        ### IMPLEMENTE A PARTIR DAQUI! Experimente algumas features! ###
        feats.append(  )
        
    return np.array(feats)

print("Extraindo features de imagem")
feats_train = extract_features(X_train)
feats_test  = extract_features(X_test)

Executando a nova Logistic Regression

In [ ]:
clf = LogisticRegression(random_state=0, solver='liblinear', multi_class='ovr')  
  
print("Treinando modelo de regressão logística")
start = time.time() ### Tempo inicial
clf.fit(feats_train, Y_train)
end = time.time()   ### Tempo inicial
print("%.2f segundos para treinar"  % (end-start) )

start = time.time() ### Tempo inicial
acc = clf.score(feats_test, Y_test)
end = time.time()   ### Tempo inicial
print("%.2f segundos para testar"  % (end-start) )

print("\nAcurácia de classificação: %.2f\n" % (acc * 100.) )